In [47]:
import pandas as pd 
import matplotlib as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.models import Model

In [48]:
user = pd.read_csv('./data/user.csv')
destination = pd.read_csv('./data/destinasi-wisata-indonesia.csv')

In [49]:
user.head()

,User_Id,Location,Age
0,1,"Semarang, Jawa Tengah",20
1,2,"Bekasi, Jawa Barat",21
2,3,"Cirebon, Jawa Barat",23
3,4,"Bekasi, Jawa Barat",21
4,5,"Lampung, Sumatera Selatan",20


In [50]:
destination.shape

(437, 14)

In [51]:
destination.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Column1,_1,Rating_Count
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,46,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-61753924,1068271528,NaN,1,18
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,46,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-61376448,1068171245,NaN,2,25
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,46,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-61253124,1068335377,NaN,3,19
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,45,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-63024459,1068951559,NaN,4,21
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,45,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-612419,106839134,NaN,5,24


In [52]:
destination['City'].value_counts()

City
Yogyakarta    126
Bandung       124
Jakarta        84
Semarang       57
Surabaya       46
Name: count, dtype: int64

In [53]:
destination.isnull().sum()

Place_Id          0
Place_Name        0
Description       0
Category          0
City              0
Price             0
Rating            0
Time_Minutes    232
Coordinate        0
Lat               0
Long              0
Column1         437
_1                0
Rating_Count      0
dtype: int64

In [54]:
destination.drop(columns=['Time_Minutes', 'Column1', '_1'], inplace=True)

In [55]:
destination.columns

Index(['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price',
       'Rating', 'Coordinate', 'Lat', 'Long', 'Rating_Count'],
      dtype='object')

In [56]:
destination.duplicated().sum()

0

In [57]:
destination.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 437 entries, 0 to 436
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Place_Id      437 non-null    int64 
 1   Place_Name    437 non-null    object
 2   Description   437 non-null    object
 3   Category      437 non-null    object
 4   City          437 non-null    object
 5   Price         437 non-null    int64 
 6   Rating        437 non-null    int64 
 7   Coordinate    437 non-null    object
 8   Lat           437 non-null    int64 
 9   Long          437 non-null    int64 
 10  Rating_Count  437 non-null    int64 
dtypes: int64(6), object(5)
memory usage: 37.7+ KB


##REKOMENDASI BY KATEGORI WISATA

### PREPROCESSING
di encode + cari kesamaan pake cosine similarity

In [58]:
oneHot = OneHotEncoder(sparse=False)

In [59]:
categoryEncoded = oneHot.fit_transform(destination[['Category']])

c:\Users\Akmal Fauzan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [60]:
categoryEncoded = pd.DataFrame(categoryEncoded, columns=oneHot.get_feature_names_out(['Category']))

In [61]:
categoryEncoded

,Category_Bahari,Category_Budaya,Category_Cagar Alam,Category_Pusat Perbelanjaan,Category_Taman Hiburan,Category_Tempat Ibadah
0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
432,0.0,1.0,0.0,0.0,0.0,0.0
433,0.0,0.0,0.0,0.0,1.0,0.0
434,0.0,0.0,0.0,0.0,1.0,0.0
435,0.0,0.0,0.0,0.0,1.0,0.0


In [62]:
cosineSim = cosine_similarity(categoryEncoded)

In [63]:
cosineSim = pd.DataFrame(cosineSim, index=destination['Place_Name'], columns=destination['Place_Name'])

In [64]:
cosineSim

Place_Name,Monumen Nasional,Kota Tua,Dunia Fantasi,Taman Mini Indonesia Indah (TMII),Atlantis Water Adventure,Taman Impian Jaya Ancol,Kebun Binatang Ragunan,Ocean Ecopark,Pelabuhan Marina,Pulau Tidung,...,Monumen Bambu Runcing Surabaya,House of Sampoerna,Atlantis Land Surabaya,Taman Hiburan Rakyat,Taman Mundu,Museum Mpu Tantular,Taman Bungkul,Taman Air Mancur Menari Kenjeran,Taman Flora Bratang Surabaya,Gereja Perawan Maria Tak Berdosa Surabaya
Place_Name,,,,,,,,,,,,,,,,,,,,,
Monumen Nasional,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Kota Tua,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Dunia Fantasi,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
Taman Mini Indonesia Indah (TMII),0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
Atlantis Water Adventure,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Museum Mpu Tantular,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Taman Bungkul,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
Taman Air Mancur Menari Kenjeran,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0


### Content-Based

In [65]:
def getRecommendations(place_name, similarity, top_n=5):

    similarityScores = similarity[place_name]
    similarPlaces = similarityScores.sort_values(ascending=False).iloc[1:top_n+1]
    
    return similarPlaces

In [66]:
#coba
recommendations = getRecommendations("Monumen Nasional", cosineSim, top_n=5)
print(recommendations)

Place_Name
Bukit Panguk Kediwung       1.0
Benteng Pendem              1.0
Monumen Palagan Ambarawa    1.0
Gedung Agung Yogyakarta     1.0
Monumen Sanapati            1.0
Name: Monumen Nasional, dtype: float64


### Collaborative Filtering

In [67]:
num_users = 1000  
num_places = len(destination)  
embedding_dim = 50

In [68]:
user_input = Input(shape=(1,), name='user_input')
place_input = Input(shape=(1,), name='place_input')

In [69]:
user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name='user_embedding')(user_input)
place_embedding = Embedding(input_dim=num_places, output_dim=embedding_dim, name='place_embedding')(place_input)

In [70]:
user_vec = Flatten()(user_embedding)
place_vec = Flatten()(place_embedding)

In [72]:
similarityScores = Dot(axes=1)([user_vec, place_vec])

In [73]:
model = Model(inputs=[user_input, place_input], outputs=similarityScores)
model.compile(optimizer='adam', loss='mse')

In [76]:
user_ids = np.array([1, 2, 3, 4])  
place_ids = np.array([10, 20, 30, 40])  
ratings = np.array([4.0, 5.0, 3.0, 4.5])

In [77]:
model.fit([user_ids, place_ids], ratings, epochs=10, verbose=1)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 17.5254
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 17.5053
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 17.4853
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 17.4651
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 17.4448
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 17.4244
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 17.4038
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 17.3830
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 17.3620
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 17.3407


In [78]:
user_id = np.array([1])  
place_id = np.array([10])

In [80]:
predicted_score = model.predict([user_id, place_id])
print(f"skor prediksi user {user_id[0]} dan tempat {place_id[0]}: {predicted_score[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
skor prediksi user 1 dan tempat 10: 0.02365119941532612
